Este código utiliza técnicas de processamento de linguagem natural (PLN) e aprendizado de máquina para analisar textos do conjunto de dados 20 Newsgroups e agrupar palavras em clusters com base em seus vetores de características. Aqui está a explicação detalhada, passo a passo:

### 1. Importação de Bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_20newsgroups_vectorized

- numpy: Biblioteca para computação numérica e manipulação de arrays.
- matplotlib.pyplot: Utilizada para criar gráficos, embora não seja usada diretamente no código apresentado.
- pandas: Biblioteca para manipulação e análise de dados em estruturas como DataFrame.
- fetch_20newsgroups_vectorized: Função da biblioteca scikit-learn que carrega o conjunto de dados 20 Newsgroups já transformado em vetores (bag-of-words ou TF-IDF).

### 2. Carregamento e Inspeção do Conjunto de Dados

In [2]:
data = fetch_20newsgroups_vectorized(remove=('headers', 'footers', 'quotes'))
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

- Carrega o conjunto de dados 20 Newsgroups, removendo cabeçalhos, rodapés e citações. O conjunto de dados contém textos já vetorizados (TF-IDF).
- data.keys() retorna as chaves do dicionário que contém o conjunto de dados, por exemplo, data, target, e feature_names (nomes das palavras).

### 3. Separação dos Dados e Dimensão da Matriz

In [16]:
X = data['data']
X.shape

(11314, 101631)

- X contém a matriz esparsa de características dos textos (data['data']), onde cada linha é um documento e cada coluna representa uma palavra (ou característica).
- X.shape retorna a forma da matriz, com a quantidade de documentos e palavras no conjunto de dados.

### 4. Contagem de Palavras

In [30]:
df_word_count = pd.DataFrame(X.sum(axis=0).T, index=data['feature_names'])
good_words = df_word_count[0].sort_values(ascending=False).iloc[:20000].index

- X.sum(axis=0) soma as contagens de todas as palavras em todos os documentos.
- A soma é transposta (.T) e um DataFrame é criado com as palavras como índice.
- As 20.000 palavras mais frequentes são selecionadas e armazenadas em good_words.

### 5. Redução Dimensional com SVD

In [4]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=10, random_state=42)

U_times_S = svd.fit_transform(X)
S = svd.singular_values_
Vt = svd.components_

U_times_S.shape, S.shape, Vt.shape

((11314, 10), (10,), (10, 101631))

- TruncatedSVD: Método de decomposição que reduz a dimensionalidade dos dados, similar à PCA, mas funciona com matrizes esparsas.
- svd.fit_transform(X): Aplica a decomposição em X, retornando a multiplicação das matrizes U (documentos) e S (valores singulares).
- S: Valores singulares da decomposição.
- Vt: Componentes que correspondem às palavras no espaço de características reduzido.
- U_times_S.shape, S.shape, e Vt.shape: Verifica as formas das matrizes resultantes da decomposição. U_times_S contém a projeção dos documentos no novo espaço de características, S tem os valores singulares, e Vt representa as palavras no novo espaço de componentes.

**Exercicio** Qual o significado de:

(a) Uma linha de `U_times_S`?

(b) Uma coluna de `Vt`

(c) Uma *coluna* de `U_times_S`?

(d) Uma *linha* de `Vt`?

(e) Os valores de `S`?

**Solução**

(a) Representação aproximada do conteudo de um documento == "id" do documento.

(b) Representação de uma palavra.

(c) Uma nivel de detalhe do corpus inteiro.

(d) Um nivel de detalhe do significado das palavras.

(e) As importancias de cada nível de detalhe. 

### 6. Agrupamento de Palavras com K-Means

In [5]:
from sklearn.cluster import KMeans
clusterer = KMeans(n_clusters=20, random_state=42)

# Como Vt tem dimensões (n_components, n_palavras), precisamos transpor.
V = Vt.T

df_V = pd.DataFrame(V, index=data['feature_names'])
df_V = df_V.div(df_V.apply(np.linalg.norm, axis=1), axis=0)

- KMeans: Algoritmo de agrupamento que forma 20 clusters (n_clusters=20), distribuindo as palavras com base em seus vetores de características.
- V é a transposta de Vt, transformando-a em uma matriz onde cada linha representa uma palavra (em vez de componentes).
- df_V cria um DataFrame com V, onde as linhas são palavras. Em seguida, normaliza os vetores de palavras para que o comprimento (norma) de cada vetor seja 1.

In [9]:
clusterer.fit(df_V)

KMeans(n_clusters=20, random_state=42)

- Aplica o K-Means para agrupar as palavras em 20 clusters.

### 7. Criação do DataFrame de Clusters

In [11]:
df = pd.DataFrame({
    'word': data['feature_names'],
    'cluster': clusterer.labels_,
})

df['cluster'].value_counts()

cluster
4     8840
8     7020
1     6983
5     6293
14    6106
7     5658
19    5152
2     5060
11    4899
0     4884
13    4758
18    4728
16    4468
10    4242
17    4146
12    4001
3     3831
15    3811
9     3634
6     3117
Name: count, dtype: int64

- Cria um DataFrame com as palavras e seus respectivos clusters.
- df['cluster'].value_counts() conta quantas palavras pertencem a cada cluster.

### 8. Distâncias de Palavras aos Clusters

In [13]:
df_distances = pd.DataFrame(
    clusterer.transform(df_V),
    index=data['feature_names'],
)

df_distances

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
00,0.560492,1.025416,1.168969,1.377959,1.304439,1.285858,1.136710,0.454621,1.291081,1.182945,1.198947,0.960432,1.359697,0.962260,0.815522,1.146212,1.185086,1.440446,0.952837,1.255574
000,0.948902,0.896819,0.685934,1.166192,1.329607,1.154961,1.340965,0.917756,1.364949,1.018251,1.231149,1.043145,1.178419,1.195667,0.628417,1.294263,1.232069,1.229038,1.187283,1.252093
0000,0.939649,1.546745,1.204887,1.403809,1.524793,1.404985,1.061367,1.175291,1.271916,0.851759,1.108676,1.199062,1.085327,1.119034,1.427594,0.773015,1.137855,1.541121,1.543623,1.462054
00000,0.835602,1.637309,1.275744,1.328179,1.394276,1.389731,0.828891,1.121569,1.582750,1.022134,1.243593,1.129452,1.041157,0.958150,1.570395,0.690254,1.170369,1.343305,1.593140,1.720831
000000,0.657184,1.026079,1.117082,1.543906,1.006299,1.009237,1.309876,1.103442,1.416260,1.143527,0.937324,0.516046,1.228445,0.779833,1.189628,0.820741,1.186531,1.222220,1.154382,1.408044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzzoh,1.104676,1.053346,1.193036,1.496200,0.197937,1.331321,1.214215,1.182746,1.396808,1.460067,1.110612,1.088254,1.499131,0.627060,1.293888,1.112939,1.257026,1.019523,0.913883,1.190385
zzzzzz,1.260217,1.740825,1.494897,0.983961,1.709040,1.328603,0.640909,1.186568,1.362261,1.011845,1.270497,1.407751,0.995955,1.389732,1.688765,1.152020,1.279425,1.353038,1.673180,1.670196
zzzzzzt,0.798010,1.467738,1.321821,1.486993,1.286152,1.045942,1.104197,1.292188,1.450998,1.062821,1.177643,0.752978,1.074773,0.883426,1.595898,0.580896,1.006832,1.273155,1.538068,1.649999
³ation,1.104195,0.877424,0.608537,1.190866,1.354811,1.267956,1.400162,0.936934,1.313293,0.988908,1.200426,1.207888,1.295401,1.246776,0.478263,1.413981,1.286311,1.337871,1.186850,1.140723


- Cria um DataFrame df_distances com as distâncias de cada palavra a todos os clusters.
- clusterer.transform(df_V) retorna as distâncias da palavra para o centroide de cada cluster.

### 9. Listar as Palavras mais Próximas de Cada Cluster

In [14]:
for cluster in range(20):
    print(f'Cluster {cluster}:')
    for word in df_distances[cluster].sort_values().head(20).index:
        print(f'   {df_distances[cluster][word]}: {word}')


Cluster 0:
   0.2886080113319972: gpc
   0.2890705320013189: teapot
   0.29025423558901986: 215
   0.29223452665166433: cyberware
   0.29223452665166433: ahpcrc
   0.29223452665166433: spaceball
   0.29223452665166433: clrview
   0.2934108688827475: wavefront
   0.2952501076789915: visualizer
   0.2952501076789915: softlab
   0.29735858100285595: avs
   0.29988108331817853: polyview
   0.29988108331817853: swedishchef
   0.29988108331817853: 80301
   0.29988108331817853: datafiles
   0.29988108331817853: omicron
   0.29988108331817853: scivi
   0.3025555405051017: gvl
   0.3025555405051017: icol
   0.3027042098511992: idl
Cluster 1:
   0.2101197085761219: 0_
   0.23858319935805694: pne
   0.24308261994532518: y0w
   0.24346731725513937: 1z6ei
   0.2474972336384418: z6e
   0.24794390308791966: bizw
   0.24924467946855913: zdk
   0.25129560354795505: m9f9fq
   0.2515395133791109: 6e1
   0.25368781645022614: 9f9f9f9f
   0.25633940223466645: bql
   0.25652292695037854: pnei4
   0.258044115

- Para cada cluster, ordena as palavras pela distância ao centroide do cluster e exibe as 20 palavras mais próximas, junto com suas distâncias.

### 10. Resumo

Este código carrega o conjunto de dados 20 Newsgroups, realiza uma redução dimensional com TruncatedSVD, agrupa palavras em 20 clusters usando K-Means, e exibe as palavras mais próximas dos centroides de cada cluster. A normalização dos vetores de características garante que a distância euclidiana seja uma medida coerente ao aplicar o K-Means.

### 11. Exercícios

#### 11.1. O que cada cluster significa?


A tarefa de clustering, como a realizada no código com o algoritmo K-Means, é uma técnica de aprendizado não supervisionado. Isso significa que os clusters são formados automaticamente com base em padrões e sem rótulos pré-definidos. A interpretação de "o que cada cluster significa" depende de como as palavras foram agrupadas e dos padrões que o algoritmo encontrou nos dados. Em geral, o significado de cada cluster precisa ser inferido analisando as palavras que estão associadas a ele.

Como interpretar os clusters:

- Agrupamento semântico: O K-Means tenta agrupar palavras com base em suas características vetoriais (neste caso, a decomposição SVD de seus vetores de frequências). Isso frequentemente resulta em grupos de palavras que aparecem em contextos semelhantes nos textos do conjunto de dados. Assim, cada cluster pode representar um tema ou tópico recorrente nos documentos.

- Tópicos semelhantes: As palavras agrupadas em um cluster geralmente compartilham uma semântica similar. Por exemplo, um cluster pode agrupar palavras relacionadas a esportes, outro a tecnologia, outro a política, etc. A interpretação pode ser feita observando as palavras mais próximas do centroide do cluster (como mostrado no código).

Como descobrir o que cada cluster significa:

Para entender o significado de cada cluster, você pode examinar as palavras que pertencem a ele, especialmente as mais próximas do centroide, que são aquelas que mais contribuem para definir o tema do cluster. Vamos revisar os passos:

Passo 1: Examinar as palavras mais próximas do centroide

No código, o trecho abaixo imprime as 20 palavras mais próximas de cada cluster:

In [ ]:
for cluster in range(20):
    print(f'Cluster {cluster}:')
    for word in df_distances[cluster].sort_values().head(20).index:
        print(f'   {df_distances[cluster][word]}: {word}')

Esse trecho imprime as palavras com as menores distâncias em relação ao centroide de cada cluster. Palavras com menores distâncias têm características vetoriais mais próximas, o que sugere que elas compartilham uma conexão semântica forte.

Passo 2: Análise qualitativa

Agora que temos as palavras mais próximas, o próximo passo é realizar uma análise qualitativa. Ao ler essas palavras, você tenta identificar padrões ou tópicos comuns. Por exemplo:

- Se um cluster contém palavras como "jogo", "gol", "time", e "campeonato", você pode inferir que esse cluster representa o tema esportes.
- Se outro cluster contém palavras como "computador", "internet", "software", e "rede", ele pode estar relacionado ao tema tecnologia.

Desafios e limitações:

- Subjetividade: A interpretação dos clusters pode ser subjetiva, pois o significado de uma palavra pode variar dependendo do contexto em que é usada.
- Dimensionalidade: Mesmo com a redução dimensional, palavras podem ter múltiplos significados ou associações que não são facilmente capturadas, o que pode fazer com que clusters incluam palavras que, à primeira vista, não parecem pertencer ao mesmo tópico.
- Números de clusters: O número de clusters (n_clusters=20 no código) também influencia os resultados. Se escolhermos um número muito pequeno ou grande, os clusters podem se tornar vagos ou muito específicos.

Conclusão:

Cada cluster provavelmente representa um tema ou tópico latente nos textos do conjunto de dados, mas o significado exato de cada cluster só pode ser interpretado analisando as palavras que compõem esse cluster. Essas palavras refletem padrões semânticos comuns, como assuntos frequentes ou contextos de uso semelhantes nos documentos.

#### 11.2. Qual o melhor número de clusters? Como identificar?

Determinar o melhor número de clusters para um conjunto de dados é uma questão importante no aprendizado de máquina e pode ser feita através de várias técnicas e métodos. A escolha do número ideal de clusters pode influenciar significativamente os resultados do seu modelo. Aqui estão algumas abordagens comuns para identificar o número apropriado de clusters:

![](photos/1_newsgroups.png)

![](photos/2_newsgroups.png)

![](photos/3_newsgroups.png)

![](photos/4_newsgroups.png)

![](photos/5_newsgroups.png)

Implementação de Exemplos

Método do Cotovelo

Aqui está um exemplo de como implementar o método do cotovelo em Python:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Range de valores de k
inertia = []
k_values = range(1, 31)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_V)
    inertia.append(kmeans.inertia_)

plt.plot(k_values, inertia, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Soma dos Quadrados das Distâncias (SSE)')
plt.show()

Silhouette Score

Para calcular o Silhouette Score:

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []

for k in k_values[1:]:  # Começa a partir de 2 para evitar erro
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_V)
    silhouette_scores.append(silhouette_score(df_V, kmeans.labels_))

plt.plot(k_values[1:], silhouette_scores, marker='o')
plt.title('Silhouette Score')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Silhouette Score')
plt.show()

Conclusão

A escolha do número de clusters é uma tarefa crítica e muitas vezes requer uma combinação de métodos para obter uma decisão informada. Testar diferentes valores de 
𝑘
k e utilizar métricas como o método do cotovelo e o Silhouette Score pode fornecer uma boa indicação do número ideal de clusters para o seu conjunto de dados.